Data Source: http://vision.ucsd.edu/~iskwak/ExtYaleDatabase/ExtYaleB.html

In [1]:
import numpy as np  
import pandas as pd  
import matplotlib.pyplot as plt  
from PIL import Image
from scipy.io import loadmat  
from scipy import stats  
from scipy.stats import multivariate_normal
import re
import glob
from operator import itemgetter 
import random
%matplotlib inline

In [ ]:
def read_images(data_path,target_folders,label_1_folder,reduce_height = 24,reduce_width = 21):
    """
    This function reads in all images inside the specified folders, and label the images based on label_1_folder
    data_path: the path of the folder where all the image folders reside in
    target_folders: the target_folders to be read from
    label_1_folder: images in the specified folders will be labeled with 1
    """
    # label_1_folder = [9,21]
    folder_paths = glob.glob(data_path + "*")
    images = [] # Initialize a list to record images
    labels = [] # Initialize a list to record labels
    for folder_path in folder_paths:
        index = int(folder_path[-2:]) # Get the index embeded in the folder path
        if index in target_folders:
            # Assign labels
            if index in label_1_folder:
                label =1
            else:
                label = 0

            # Read in images and corresponding labels
            img_paths = glob.glob(folder_path + "/*.pgm")
            for img_path in img_paths: 
                if img_path.find("Ambient")>0:
                    img_paths.remove(img_path) # We do not want the "Ambient" image because it is a profile picture
                else:
                    # img = plt.imread(img_path) # Used to read image without resizing
                    img_raw = Image.open(img_path) # Used when we need to resize the image (downsize in this case)
                    img_reduce = img_raw.resize((reduce_width, reduce_height), Image.BILINEAR) # Resize the image
                    img = np.array(img_reduce) # This step is necessary if we use Image.open()
                    images.append(img)
                    labels.append(label)
    return images,labels


def dark_pixel_curve(images,light_threshold = 20):
    """
    Images are taken at different lighting conditions; thus some of the photos are dark. In order to avoid 
    the impact of the bad lighting conditions, we need to remove photos with large number of dark pixels. 
    This curve shows us the number of images to be removed at different thresholds (total number of pixels 
    that are below 20 in one image). It can help us select an appropriate threshold. 
    """
    height, width = images[0].shape # Get the dimension of one image
    images_num = len(images)
    thresh_list = range(100,height*width,100) # Threshold levels to be tested: from 100 to the total pixels
    remove_list = []
    for dark_pixel_threshold in thresh_list:
        remove_count = 0
        for i in range(0,images_num):
            if sum(sum(images[i] < light_threshold)) > dark_pixel_threshold:
                remove_count = remove_count + 1
        remove_list.append(remove_count)
    
    plt.plot(thresh_list,remove_list)
    plt.xlabel("Number of dark pixels in an image")
    plt.ylabel("Number of images to be removed from the list")
    plt.title("Select the right threshold level")
    
def remove_dark_img(imgs,labels,dark_pixel_threshold,light_threshold = 20):
    """
    This function remove images that have more dark pixels (<20) than our threshold
    """
    remove_count = 0
    imgs_num = len(imgs)
    for i in range(imgs_num-1,0-1,-1):
        if sum(sum(imgs[i] < light_threshold)) > dark_pixel_threshold:
            del imgs[i]
            del labels[i]
            remove_count = remove_count + 1
    print (remove_count,' images are above our threshold and thus removed from the list')
    return imgs,labels,remove_count

def plot_images(imgs,labels):
    """
    Plot 25 images selected randomly
    """
    ind = np.random.permutation(len(imgs))

    # Create figure with 5x5 sub-plots.
    fig, axes = plt.subplots(5, 5,figsize=(15,15))
    fig.subplots_adjust(hspace=0.1, wspace=0.01)

    for i, ax in enumerate(axes.flat): 
        ax.imshow(imgs[ind[i]], plt.cm.gray)
        xlabel = "Anomaly: {0}".format(labels[ind[i]])
        # Show the classes as the label on the x-axis.
        ax.set_xlabel(xlabel)
        
        # Remove ticks from the plot.
        ax.set_xticks([])
        ax.set_yticks([])
    
    plt.show()

def show_anomaly_images(images,labels):
    """
    This function randomly show 9 images with label 1
    """
    anomaly_label_index = np.asarray(np.where(labels)).reshape(-1) # Get the indice of anomaly
    anomaly_image = [images[i] for i in anomaly_label_index] # Extract the images labeled as anomaly
    anomaly_label = [labels[i] for i in anomaly_label_index] # Extract the images labeled as anomaly
    plot_images(anomaly_image,anomaly_label) # Show 9 images randomly
    
def plot_eigenfaces(pca_matrix,height, width):
    """
    This function plot the eigenfaces based on the given PCA Matrix
    """
    n_eigen = pca_matrix.shape[1]
    # Define the layout of the plots
    n_row = 4
    n_col = n_eigen//n_row 

    # Create figure with 3x3 sub-plots.
    fig, axes = plt.subplots(n_row, n_col,figsize=(15,15))
    fig.subplots_adjust(hspace=0.1, wspace=0.01)

    for i, ax in enumerate(axes.flat): 
        ax.imshow(pca_matrix[:,i].reshape(height, width), plt.cm.gray)
        xlabel = "Eigenface: {0}".format(i+1)
        # Show the classes as the label on the x-axis.
        ax.set_xlabel(xlabel)
        
        # Remove ticks from the plot.
        ax.set_xticks([])
        ax.set_yticks([])

def mean_shift(components):
    """
    This function applies mean shift to each component in the component matrix
    The input components is a n*m matrix. Each row correspons to one component.
    It is important to return the components' mean vector: we will need it in PCA reconstruction
    """
    component_mean = np.mean(components,axis = 1)
    shifted_components = (components.T - component_mean).T # Necessary to take transpose twice here
    return shifted_components, component_mean

def check_eigen(eigen_value, eigen_vector,cov_matrix):
    """
    This function check the correctness of eigenvector & eigenvalue through the equation
    cov_matrix * eigen_vector = eigen_value * eigen_vector
    """
    for i in range(len(eigen_value)): 
        n = cov_matrix.shape[1]
        eigv = eigen_vector[:,i].reshape(1,n).T 
        np.testing.assert_array_almost_equal(cov_matrix.dot(eigv), eigen_value[i] * eigv, decimal=6, err_msg='', verbose=True)
        
def plot_compare_after_reconst(img_matrix_reconst,imgs_matrix,height,width):
    """
    This function compares the images reconstructed after PCA with their original one.
    The shape of both image matrice in the input is n*m, where n is the number of components, 
    and m is the number of images.
    """
    # Permutate through the image index
    ind = np.random.permutation(imgs_matrix.shape[1])

    # Create figure with multiple sub-plots.
    fig, axes = plt.subplots(4, 4,figsize=(15,15))
    fig.subplots_adjust(hspace=0.1, wspace=0.01)

    # Initialize the counter of images
    image_count = 0 

    for i, ax in enumerate(axes.flat): 
        if i % 2 == 0:
            image_count += 1
            ax.imshow(imgs_matrix[:,ind[i]].reshape(height,width), plt.cm.gray)
            xlabel = "Example {0}: Original Image".format(image_count)
        else:
            ax.imshow(img_matrix_reconst[:,ind[i-1]].reshape(height,width), plt.cm.gray)
            xlabel = "Example {0}: Reconstructed from PCA".format(image_count)
        # Show the classes as the label on the x-axis.
        ax.set_xlabel(xlabel)

        # Remove ticks from the plot.
        ax.set_xticks([])
        ax.set_yticks([])

    plt.show()

def split_training(labels,ratio):
    """
    This function Split the Data into the Training and Validation Set. 
    Its output is the indice of images to be assigned to the Training/Validation Set. 
    The input "labels" is a hvector
    The ratio is a number between [0,1] that represents the percentage of images to be assigned to the training set
    """
    ratio = 0.8 # The percentage of images to be splitted into the training set
    m = len(labels)
    training_size = int(m*ratio)
    while 1:
        ind = np.random.permutation(m) # Permutate to generate random indice within m
        train_ind = ind[:training_size-1]
        test_ind = ind[training_size:]
        # if (sum(itemgetter(*train_ind)(labels)) > 0 and sum(itemgetter(*test_ind)(labels)) > 0):
        if (sum(labels[train_ind]) > 0 and sum(labels[test_ind]) > 0):
            break
    return train_ind, test_ind

def estimate_gaussian(X):
    """
    Compute the parameters of the Gaussian Distribution
    Note: X is given in the shape of m*k, where k is the number of (reduced) dimensions, and m is the number of images
    """
    mu =np.mean(X,axis=0)
    cov = np.cov(X,rowvar=0)

    return mu, cov

def fit_multivariate_gaussian(data):
    """
    This function is used to compute the mu and cov based on the given data, and fit a multivariate gaussian dist
    This data is given as a m*k matrix, where m represents the number of samples, and k represents the number of dimensions
    """
    mu, cov = estimate_gaussian(data)
    dist = multivariate_normal(mean = mu, cov = cov)
    return dist

def select_threshold(pval, yval, p_anomaly_switch = 0):  
    """
    This function finds the best threshold value to detect the anomaly given the PDF values and True label Values
    pval: Probability based on the Multivariate Gaussian Distribution
    yval: True label value
    p_anomaly_switch: 1 if the given pval is the probability of being an anomaly, otherwise that is the probability
    of not being an anmaly. By default it is 0
    """
    best_epsilon = 0
    best_f1 = 0
    best_tp = 0
    best_fp = 0
    best_fn = 0

    step = (pval.max() - pval.min()) / 1000

    for epsilon in np.arange(pval.min(), pval.max(), step):
        # If the given p is the probability of being an anomaly, we need to find the threshold so that
        # if p > threshold, it will be an anomaly
        # IF the given p is hte probability of not being an anomaly, we need to find the threshold so that 
        # if p < threshold, it will be an anomaly
        if p_anomaly_switch == 1:
            preds = pval > epsilon
        else:
            preds = pval < epsilon

        tp,tn,fp,fn,f1 = eval_prediction(preds,yval)

        if f1 > best_f1:
            best_f1 = f1
            best_epsilon = epsilon
            best_tp = tp
            best_fp = fp
            best_fn = fn
    #return best_epsilon, best_f1, best_tp, best_fp, best_fn
    return best_epsilon

def eval_prediction(pred,yval,rate = False):
    """
    Function to evaluate the correctness of the predictions with multiple metrics
    If rate = True, we will return all the metrics in rate (%) format (except f1)
    """
    true_positive = np.sum(np.logical_and(pred == 1, yval == 1)).astype(float) # True Positive
    true_negative = np.sum(np.logical_and(pred == 0, yval == 0)).astype(float) # True Negative
    false_positive = np.sum(np.logical_and(pred == 1, yval == 0)).astype(float) # False Positive
    false_negative = np.sum(np.logical_and(pred == 0, yval == 1)).astype(float) # False Negative

    precision = true_positive / max(1,true_positive + false_positive)
    recall = true_positive / max(1,true_positive + false_negative)
    f1 = (2 * precision * recall) / max(1,precision + recall)
    # A more direct version of f1 is f1 = 2*tp/(2*tp+fn+fp)
    
    if rate:
        n_p = sum(yval == 1)     # Number of Positive
        n_n = yval.shape[0] - n_p # Number of Negative
        tpr = true_positive/n_p
        tnr = true_negative/n_n
        fpr = false_positive/n_n
        fnr = false_negative/n_p
        return tpr,tnr,fpr,fnr,f1
    else:
        return true_positive,true_negative,false_positive,false_negative,f1
    
def compute_pca_matrix(data, n_components):
    """
    This function compute the pca matrix with the given data
    The data should be given in the matrix form n*m, where n is the number of dimensions, and m is the number of samples
    The data should have been processed with mean-shift
    n_components: Number of components to be kept after PCA
    """
    # Compute the Covariance Matrix
    cov_matrix = np.cov(data)

    # Compute the eigen value and eigen vectors
    eigen_value, eigen_vector = np.linalg.eig(cov_matrix)

    # Sort the eigenvectors by decreasing eigenvalues
    # First make a list of (eigenvalue, eigenvector) tuples 
    eig_pairs = [(np.abs(eigen_value[i]), eigen_vector[:,i]) for i in range(len(eigen_value))] 
    # Sort the (eigenvalue, eigenvector) tuples from high to low 
    eig_pairs.sort(key=lambda x: x[0], reverse=True)

    # Convert the sorted eigen vector list to matrix form
    eigen_vector_sorted = np.zeros((height*width,n_components))
    for i in range(0,n_components):
        eigen_vector_sorted[:,i] = eig_pairs[i][1]

    # Cut the sorted eigenvectors by columns to get the transformational matrix for PCA
    pca_matrix = eigen_vector_sorted[:,:n_components]

    return pca_matrix

def find_euclidean_distance(matrix1,matrix2):
    """
    This function find the Euclidean Distance between two Matric
    The distance is between the same columns of two matric
    """
    dist = np.linalg.norm(matrix1 - matrix2,axis = 0) # By specifying axis = 0, we find the distance between columns
    return dist

def select_threshold_distance(edistance, yval):  
    """
    This function finds the best threshold value to detect the anomaly given the PDF values and True label Values
    edistance: euclidean distance 
    yval: True label value
    """
    best_epsilon = 0
    best_f1 = 0
    best_tpr = 0
    best_tnr = 0
    best_fpr = 0
    best_fnr = 0

    step = (edistance.max() - edistance.min()) / 1000

    for epsilon in np.arange(edistance.min(), edistance.max(), step):
        preds = edistance > epsilon
        
        tpr,tnr,fpr,fnr,f1 = eval_prediction(preds,yval,rate = True)

        if f1 > best_f1:
            best_f1 = f1
            best_epsilon = epsilon
            best_tpr = tpr
            best_tnr = tnr
            best_fpr = fpr
            best_fnr = fnr
    return best_epsilon,best_tpr,best_tnr,best_fpr,best_fnr,best_f1

## Read in Images and Remove Over-shadowed Ones

In [ ]:
# Define the images to be read and the corresponding labels
label_1_folder = [9,21]
target_folders = range(1,22)
data_path = "Yale Face B/CroppedYale/"

# We also need to reduce the size of the image for the convenience of computation
reduce_height = 24
reduce_width = 21

# Read the images and reduce the size
images,labels = read_images(data_path,target_folders,label_1_folder,reduce_height,reduce_width)

# Number of labels as "Anomaly" and Total Number of Labels
sum(labels),len(labels)

In [ ]:
# To evaluate the threshold of the dark pixels
# dark_pixel_curve(images)

imgs = images[:] # Create a copy
# Eliminate the images and labels whose number of dark pixels are above the threshold
# The threshold is determined based on the dark_pixel_curve() function above
imgs,labels,remove_count = remove_dark_img(imgs,labels,180) 

plot_images(imgs,labels)

In [ ]:
# Randomly select and show anomalous images
show_anomaly_images(imgs,labels)

## Apply PCA for Dimension Reduction

In [ ]:
# Define the number of Principal Components we want to keep from the image
n_components = 20

# Find the dimension of one image
height, width = imgs[0].shape
num_imgs = len(imgs)
height, width,num_imgs

### Convert the Images List to a 2-Dimensional Matrix
Convert each image from 2D to 1D array, and each column of the new matrix will be one image.

In [ ]:
# Initialize the matrix to store the entire image list
imgs_matrix = np.zeros((height*width,num_imgs)) 

# Iterate through each image, convert it into an array, and add to the imgs_matrix as a column
for i in range(0,len(imgs)):
    imgs_matrix[:,i] = imgs[i].reshape(height*width)
imgs_matrix.shape

### Take a Mean-Shift
We applied the Mean-Shift on each component before computing the Covariance Matrix. It is important to save the vector of the components' mean: we will use it when we reconstruct the data after we applied PCA

In [ ]:
img_matrix_shifted, component_mean = mean_shift(imgs_matrix)

### Compute the Covariance Matrix of the Image Matrix
The Covariance Matrix should be a symmetric square matrix with shape $n*n$, where $n$ is the row number of the Image Matrix, or **the number of the components**.

In [ ]:
cov_matrix = np.cov(img_matrix_shifted)
cov_matrix.shape

### Compute the eigenvectors and the corresponding eigenvalues based on the Covariance Matrix

In [ ]:
# Compute the eigen value and eigen vectors
eigen_value, eigen_vector = np.linalg.eig(cov_matrix)

# Sort the eigenvectors by decreasing eigenvalues
# First make a list of (eigenvalue, eigenvector) tuples 
eig_pairs = [(np.abs(eigen_value[i]), eigen_vector[:,i]) for i in range(len(eigen_value))] 
# Sort the (eigenvalue, eigenvector) tuples from high to low 
eig_pairs.sort(key=lambda x: x[0], reverse=True)

# Convert the sorted eigen vector list to matrix form
eigen_vector_sorted = np.zeros((height*width,n_components))
for i in range(0,n_components):
    eigen_vector_sorted[:,i] = eig_pairs[i][1]

### Compute the PCA Matrix and Visualize the Eigenfaces

In [ ]:
# Cut the sorted eigenvectors by columns to get the transformational matrix for PCA
pca_matrix = eigen_vector_sorted[:,:n_components]
pca_matrix.shape

In [ ]:
# Visualize the eigenfaces with the pca matrix
plot_eigenfaces(pca_matrix,height, width)

### Reconstruct the Face Images after PCA
First we applies PCA on the image data matrix to downsize its dimensions. Then we reconstruct the image matrix through the PCA matrix and add the mean of each component back. Finally we plot several examples to compare the original face images with the reconstructed face images. 

In [ ]:
# Compute the transformed image
# Shape of pca_matrix: n * k
# Shape of imgs_matrix: n * m
# Shape of the transformed face image matrix: k * m
img_pca_tranf = pca_matrix.T.dot(img_matrix_shifted)

# Reconstruct through PCA Matrix and Mean Vector
# Shape of the reconstructed face image matrix: n * m
# component_mean is a vector and we need to convert it to a one-column matrix for the addition
img_matrix_reconst = pca_matrix.dot(img_pca_tranf) + component_mean.reshape(height*width,1)

# Plot the original images and their reconstructed version for comparison
plot_compare_after_reconst(img_matrix_reconst,imgs_matrix,height,width)

## Anomaly Detection
We will use two methods of the Anomaly Detection. 
1. We will use the probability based on the Multivariate Gaussian Distribution.
2. We will use the Reconstruction Error Method

### Split the Data into the Training and Validation Set
With the function, we get the indice of images to be assigned to the training and the validation set

In [ ]:
# Vectorize the labels list
labels_vector = np.hstack(labels) # Easier to get multiple items from a vector than from a list

# Split the images and labels
ratio = 0.8 # 80% of the images will be splitted to the training set
train_ind, test_ind = split_training(labels_vector,ratio)

imgs_train = img_pca_tranf[:,train_ind]
imgs_test = img_pca_tranf[:,test_ind]
labels_train = labels_vector[train_ind]
labels_test = labels_vector[test_ind]

imgs_train.shape, imgs_test.shape, labels_train.shape,labels_test.shape

### Method 1: Multivariate Gaussian Distribution

In [ ]:
# We need to separete the data to get different Gaussian Distribution Model
imgs_train_anomaly = imgs_train[:,labels_train == 1]
imgs_train_normal = imgs_train[:,labels_train == 0]

# Get Gaussian Distribution Model from the Anomaly Data and Normal Data Separately
# Note: fit_multivariate_gaussian() is my own coded function
dist_anomaly = fit_multivariate_gaussian(imgs_train_anomaly.T)
dist_normal = fit_multivariate_gaussian(imgs_train_normal.T)

# Get Probability of being Anomaly vs. being Normal
p_train_anomaly = dist_anomaly.pdf(imgs_train.T) # Probability of Being Abnormal
p_train_normal = dist_normal.pdf(imgs_train.T)   # Probability of Being Normal

# Train the Anomaly Detector
threshold_anomaly = select_threshold(p_train_anomaly, labels_train, p_anomaly_switch = 1)
threshold_normal  = select_threshold(p_train_normal, labels_train, p_anomaly_switch = 0)

In [ ]:
# Case 1: Predict with only the threshold at the probability of being anomaly
preds1 = p_train_anomaly > threshold_anomaly
tpr,tnr,fpr,fnr,f1 = eval_prediction(preds1,labels_train,rate = True)
tpr,tnr,fpr,fnr,f1

In [ ]:
# Case 2: Predict with only the threshold at the probability of being normal
preds2 = p_train_normal < threshold_normal
tpr,tnr,fpr,fnr,f1 = eval_prediction(preds2,labels_train,rate = True)
tpr,tnr,fpr,fnr,f1

In [ ]:
# Case 3: Predict as Anomaly if one of the detector claims as Anomaly
np.logical_or(p_train_anomaly > threshold_anomaly, p_train_normal < 0)
# preds3 = (p_train_anomaly < threshold_anomaly)*(p_train_normal > threshold_normal)
preds3 = np.logical_or(p_train_anomaly > threshold_anomaly, p_train_normal < threshold_normal)
tpr,tnr,fpr,fnr,f1 = eval_prediction(preds3,labels_train,rate = True)
tpr,tnr,fpr,fnr,f1

In [ ]:
# Case 4: Predict as Anomaly if both of the detectors claim as Anomaly
preds4 = np.logical_and(p_train_anomaly > threshold_anomaly, p_train_normal < threshold_normal)
tpr,tnr,fpr,fnr,f1 = eval_prediction(preds4,labels_train,rate = True)
tpr,tnr,fpr,fnr,f1

From the examination above, we can find the detector of being anomaly alone perform much better than the detector of being abnormal.

#### Apply on the Test Set

In [ ]:
p_test_anomaly = dist_anomaly.pdf(imgs_test.T)   # Probability of Being Normal
preds_test = p_test_anomaly > threshold_anomaly
tpr,tnr,fpr,fnr,f1 = eval_prediction(preds_test,labels_test,rate = True)
tpr,tnr,fpr,fnr,f1

### Method 2: PCA Reconstruction Error
The method will measure the Euclidean distance between each data point on the image and its new point after reconstruction from the PCA, whcih was generated sololy with the Normal Data. The basis of this method is that the Anomalies will give large values of distance. I will use the training data to find the optimal threshold to identify the Anomalies. 

#### Generate a new PCA sololy with the Normal Data
Since we need to re-run a new PCA, we need to go back to the place where we took a Mean Shift on the Image Matrix and start over again

In [ ]:
# Isolate the Normal Data from the Training Set
# Note: we use the same index but on the img_matrix_shifted
img_matrix_shifted_train = img_matrix_shifted[:,train_ind]
img_matrix_shifted_train_normal = img_matrix_shifted_train[:,labels_train == 0]

pca_matrix_normal = compute_pca_matrix(img_matrix_shifted_train_normal, n_components)

# Visualize the new eigenfaces
#plot_eigenfaces(pca_matrix_normal,height, width)

#### Reconstruct the Data with the new PCA Matrix
Use the PCA on both the Normal and Anomalous Data

In [ ]:
img_pca_tranf_new = pca_matrix_normal.T.dot(img_matrix_shifted) # Apply on all the data

# Reconstruct through PCA Matrix and Mean Vector
img_matrix_reconst_new = pca_matrix_normal.dot(img_pca_tranf_new) + component_mean.reshape(height*width,1)

#### Find the Optimal Threshold based on the training dataset

In [ ]:
# Find the euclidean distance between the reconstructed dataset and the original ()
dist = find_euclidean_distance(img_matrix_reconst_new,imgs_matrix)
np.min(dist),np.mean(dist),np.max(dist)

In [ ]:
dist_train= dist[train_ind]
threshold2, tpr,tnr,fpr,fnr,f1 = select_threshold_distance(dist_train, labels_train)
tpr,tnr,fpr,fnr,f1

#### Apply on the Test Set

In [ ]:
dist_test= dist[test_ind]
preds_test2 = dist_test > threshold2
tpr,tnr,fpr,fnr,f1 = eval_prediction(preds_test2,labels_test,rate = True)
tpr,tnr,fpr,fnr,f1